In [105]:
%%capture
!pip install geopandas==0.10.2
!pip install mapclassify==2.4.3
!pip install selenium==4.2.0
!pip install pygeos==0.10.2
!pip install -U folium

In [106]:
from folium import GeoJson, GeoJsonTooltip, Map, Marker, Icon, PolyLine, FeatureGroup
from shapely.geometry import LineString
from geopy.geocoders import Nominatim
from folium.map import LayerControl
import matplotlib.pyplot as plt
from google.colab import drive
import plotly.express as px
from tqdm.auto import tqdm
import geopandas as gpd
import seaborn as sns
import pandas as pd
import numpy as np
import sys, os, re
import folium
import json

drive.mount('/content/drive')

folder_directory = "/content/drive/MyDrive/_____SHARED/Geospatial"
sys.path.append(folder_directory)
os.chdir(folder_directory)

import utils, styles
tiles = 'Stamen Terrain'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# In the Heart of Dolomities 

Prepare the multipolygon areas for the Dolomiti Mountain inside the region of Trentino - Alto Adige.

The data are from the offical website of [Dolomiti Unesco](https://www.dolomitiunesco.info/), that are available in .kml format.

In [107]:
# read kml in GeoPandas https://docs.astraea.earth/hc/en-us/articles/360043923831-Read-a-KML-File-into-a-GeoPandas-DataFrame

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
dolomiti_df = gpd.read_file('data/geo_dolomiti/dolomiti_area.kml', driver='KML')

print(f"Reference system information:\n{ '-'*50}\n{dolomiti_df.crs.__repr__()}{'-'*50}")

Reference system information:
--------------------------------------------------
<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
--------------------------------------------------


In [108]:
dolomiti_df["name"] = dolomiti_df["Name"].apply(lambda x: x.replace(',', ' &'))
dolomiti_df['url_info'] = dolomiti_df.Description.apply(lambda x: x.split('>')[1])

dolomiti_df = dolomiti_df.loc[:, ['name', 'url_info', 'geometry']]
dolomiti_df.head(2)

,name,url_info,geometry
0,Sistema 1 - Pelmo & Croda da Lago,http://www.dolomitiunesco.info/?gruppo-dolomit...,"POLYGON Z ((12.12142 46.40488 0.00000, 12.1214..."
1,Sistema 2 - Marmolada,http://www.dolomitiunesco.info/?gruppo-dolomit...,"POLYGON Z ((11.82319 46.45296 0.00000, 11.8221..."


## Overaly on Trentino-Alto adige area

Extract the 


In [109]:
italy_regions = gpd.read_file("data/istat_administrative.gpkg", layer="regions")

# Set correct crs
italy_regions = italy_regions.to_crs("EPSG:4326")

# Selecting only interested regions
selected_regions = ['Veneto', 'Trentino-Alto Adige', 'Friuli Venezia Giulia']
dolomiti_regions = italy_regions.loc[italy_regions.DEN_REG.isin(selected_regions)]

In [110]:
dolomiti_df['representative_point'] = dolomiti_df.to_crs(epsg=32632).geometry.representative_point().to_crs(epsg=4326)
dolomiti_df['representative_point'] = dolomiti_df['representative_point'].apply(str)
dolomiti_df['area'] = dolomiti_df.to_crs(epsg=32632).geometry.area.apply(lambda x: f"{int(x)} m^{2}") #metersquare since we are in 32632

In [113]:
dolomiti_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [111]:
# https://gis.stackexchange.com/questions/392531/modify-geojson-tooltip-format-when-using-folium-to-produce-a-map
# https://stackoverflow.com/questions/58305337/plotting-polygons-in-python-using-geopandas-and-folium
# https://stackoverflow.com/questions/63960271/folium-geojson-custom-color-map
# https://leafletjs.com/reference.html#path-option

dolomiti_map = utils.get_new_map(title='Dolomities in Italy', tiles=tiles)

feature_layers = dict(
  dolomiti_area = FeatureGroup(name='Area Dolomiti'),
  regions_layer = FeatureGroup(name='Italian Regions')   
)

dolomiti_region = GeoJson(
    data = dolomiti_regions,
    style_function = styles.style_region,
    tooltip = GeoJsonTooltip(fields=['DEN_REG'])
  ).add_to(feature_layers["regions_layer"])

dolomiti_geo = GeoJson(
    data = dolomiti_df,
    style_function = styles.style_dolomiti,
    tooltip = GeoJsonTooltip(
            fields=[
                'name',
                'url_info',
                'area',
                'representative_point'
            ])
  )

dolomiti_geo.add_to(feature_layers["dolomiti_area"])

utils.add_map_infos(feature_layers, dolomiti_map)

In [ ]:
italy_municipalities = gpd.read_file("data/istat_administrative.gpkg", layer="municipalities")

# Set correct crs
italy_municipalities = italy_municipalities.to_crs("EPSG:4326")

def check_geom(geom, regions_df):
  res = []
  for idx, region in regions_df.iterrows():
    geometry = region.geometry
    if (
      geom.within(geometry) or 
      geom.overlaps(geometry) or 
      geom.touches(geometry)
    ): 
      res.append(region['name'])
  return ','.join(res)
  

buffered_dolomiti = dolomiti_df.copy()
buffered_dolomiti.geometry = dolomiti_df.to_crs(epsg=32632).geometry.buffer(500).to_crs(epsg=4326).values

selected_municipalities = italy_municipalities.geometry.apply(lambda x: check_geom(x, buffered_dolomiti)) # dolomiti_df dolomiti_regions
italy_municipalities['dolomiti'] = selected_municipalities.values
dolomiti_municipalities = italy_municipalities.loc[selected_municipalities != '']
dolomiti_municipalities = dolomiti_municipalities.loc[:, ['COD_REG', 'COMUNE', 'geometry', 'dolomiti']]

dolomiti_municipalities.head(2)

,COD_REG,COMUNE,geometry,dolomiti
2760,4,Badia,"MULTIPOLYGON (((11.89993 46.63847, 11.89978 46...","Sistema 5 - Dolomiti settentrionali,Sistema 6 ..."
2763,4,Braies,"MULTIPOLYGON (((12.13557 46.73782, 12.13594 46...",Sistema 5 - Dolomiti settentrionali


In [ ]:
# add regions of the municipality
dolomiti_municipalities = dolomiti_municipalities.merge(
    dolomiti_regions.loc[:, ['COD_REG', 'DEN_REG']],
    on='COD_REG'
  )

In [ ]:
# create the map and add geojson infos
dolomiti_map_municipalities = utils.get_new_map(title='Dolomities over municipalities', tiles=tiles)

feature_layers = dict(
  dolomiti_area = FeatureGroup(name='Area Dolomiti'),
  municipalities_layer = FeatureGroup(name='Italian Municipalities')
)

dolomiti_geo.add_to(feature_layers["dolomiti_area"])

GeoJson(
    data = dolomiti_municipalities,
    style_function = styles.style_municipalities,
    tooltip = GeoJsonTooltip(
            fields=[
                'COMUNE',
                'DEN_REG',
                'dolomiti'
            ])
).add_to(feature_layers["municipalities_layer"])

utils.add_map_infos(feature_layers, dolomiti_map_municipalities)

## Best CAI Alpine HUT - Geocode names

Manually extract the CAI alpine hut and organize them into a usefull way. 

In [ ]:
from geopy.geocoders import Nominatim
from geopandas.tools import geocode

# https://www.cai.it/andare-in-montagna/rifugi-e-bivacchi/rifugi-del-club-alpino-italiano/

rifugi_trentino = pd.read_excel(
    io='data/alpine_huts/rifugi_trentino.xlsx', 
    sheet_name=0, 
    header=1
  )

rifugi_veneto = pd.read_excel(
    io='data/alpine_huts/rifugi_veneto.xlsx', 
    sheet_name=0, 
    header=1
  )

rifugi = pd.concat([rifugi_trentino, rifugi_veneto])
rifugi.head(2)

,Nome,Tipo,Sezione,Quota (m slm),Posti totali,Categoria
0,Rifugio Vioz,Rifugio custodito,S.A.T.,3535,66,E
1,Rifugio Boè,Rifugio custodito,S.A.T.,2873,69,D


In [ ]:
user_agent = "Mozilla/5.0 (Linux; Android 10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Mobile Safari/537.36"
geolocator = Nominatim(user_agent=user_agent, timeout=3)

def geocode_sat(rifugi):
  detail = []
  not_founded = []
  for idx, row in tqdm(rifugi.iterrows(), total=len(rifugi)):
    name = row['Nome']
    if name in utils.names_adj:
      name = utils.names_adj[name]

    geocodes = geolocator.geocode(
            query = name, 
            addressdetails = True,
            extratags = True
          )
    
    if geocodes:
      detail.append(geocodes.raw)
    else:
      not_founded.append(row['Nome'])

  assert len(not_founded) == 0, 'missing something'

  rifugi = utils.expand_raw_geocode(pd.DataFrame(detail))
  rifugi = gpd.GeoDataFrame(rifugi, crs="EPSG:4326")
  rifugi = rifugi.loc[~rifugi.county.isin(['Roma Capitale', "Reggio nell'Emilia"])]
  rifugi.drop(columns=['boundingbox']).to_file('data/rifugi.shp') 
  return rifugi
  
sat_rifugi = (
    gpd.read_file("data/alpine_huts/rifugi.shp") 
    if os.path.exists('data/alpine_huts/rifugi.shp')
    else geocode_sat(rifugi))

In [ ]:
sat_map = utils.get_new_map(title='Hut mantained by SAT', tiles=tiles)

unique_provice_dolomities = sat_rifugi.county.unique().tolist()
colors = ['gray', 'lightgreen', 'green', 'orange', 'darkred','black']

feature_layers = {
    group:FeatureGroup(name=f"<span style='color:{utils.c_colors[idx]}'>Hut in {group}</span>") 
    for idx, group in enumerate(unique_provice_dolomities)
  }

feature_layers["dolomiti_area"] = FeatureGroup(name='Area Dolomiti')
dolomiti_geo.add_to(feature_layers["dolomiti_area"])


for idx, row in sat_rifugi.iterrows():
  info = [f"{k}: {v}" for k,v in row.to_dict().items() if v]
  marker_group = row.county
  color = utils.c_colors[unique_provice_dolomities.index(row.county)]
  marker = Marker(
      location = [i[0] for i in reversed(row.geometry.xy)],
      popup = '<br/>'.join(info), 
      tooltip = '<br/>'.join(info),
      icon = folium.Icon(color=color)
  )

  marker.add_to(feature_layers[marker_group])


utils.add_map_infos(feature_layers, sat_map)

In [ ]:
sat_rifugi

,municipali,county,state,road_name,extended_n,opening_ho,contact_mo,winter_roo,capacity,website,email,display_na,type_descr,geometry
0,Comunità della Valle di Sole,Provincia di Trento,Trentino-Alto Adige/Südtirol,Vioz,Rifugio Mantova al Vioz,Jun 20-Sep 20,+39 339 2798826,yes,30,http://www.rifugiovioz.it/,info@rifugiovioz.it,"Rifugio Mantova al Vioz, Vioz, Peio, Comunità ...",alpine_hut,POINT (10.63578 46.39920)
1,Pustertal - Val Pusteria,Bolzano - Bozen,Trentino-Alto Adige/Südtirol,Cresta Strenta - Lichtenfelser Weg,Rifugio Boé,Jun 20-Sep 20,+39 349 5730110,yes,69,http://www.rifugioboe.it/,rifugioboe@gmail.com,"Rifugio Boé, Cresta Strenta - Lichtenfelser We...",alpine_hut,POINT (11.82328 46.51461)
2,Gemeinde Gries am Brenner,Bezirk Innsbruck-Land,Tirol,Geistbeckweg,Europahütte - Rifugio Europa,Jun-Sep,None,None,65,http://www.europahütte.it,info@europahütte.it,"Europahütte - Rifugio Europa, Geistbeckweg, Ge...",restaurant,POINT (11.58068 46.99718)
3,Comunità della Valle di Sole,Provincia di Trento,Trentino-Alto Adige/Südtirol,Fünklegrat,Rifugio Guido Larcher al Cevedale,Jun 20-Sep 20,None,yes,80,None,info@rifugiocevedale.it,"Rifugio Guido Larcher al Cevedale, Fünklegrat,...",alpine_hut,POINT (10.66605 46.43692)
4,Pustertal - Val Pusteria,Bolzano - Bozen,Trentino-Alto Adige/Südtirol,Pisciadústeig - Via Ferrata Brigata Tridentina,Utia Pisciadù - Pisciadùhütte - Rifugio Franco...,None,None,None,176,None,None,Utia Pisciadù - Pisciadùhütte - Rifugio Franco...,alpine_hut,POINT (11.82180 46.53642)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,None,Belluno,Veneto,Sentiero Sperti,Rifugio 7° Alpini,None,None,yes,68,None,None,"Rifugio 7° Alpini, Sentiero Sperti, Belluno, V...",alpine_hut,POINT (12.18878 46.22193)
89,Comunità Montana Agno-Chiampo,Vicenza,Veneto,Strada delle Siebe,None,None,None,None,None,None,None,"Rifugio Schio, Strada delle Siebe, Recoaro Ter...",yes,POINT (11.17078 45.72641)
90,Cimolais,Pordenone,Friuli-Venezia Giulia,Via Comune,None,None,None,None,None,None,None,"Casera Bosconero, Via Comune, Cimolais, Porden...",collapsed,POINT (12.39677 46.33909)
91,None,Belluno,Veneto,Sentiero della Casera della Cengia,Rifugio Furio Bianchet,None,None,None,None,None,None,"Rifugio Furio Bianchet, Sentiero della Casera ...",alpine_hut,POINT (12.16107 46.24348)
